In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO

In [24]:
url = 'https://transparencia.alesc.sc.gov.br/deputados.php?ano=2025&mes=9'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
tabela = soup.find('table', class_='pagamentos')
df = pd.read_html(StringIO(str(tabela)))[0]

In [1]:
import pandas as pd
import requests
from io import StringIO
from concurrent.futures import ThreadPoolExecutor
import time
import random
from bs4 import BeautifulSoup
from tqdm import tqdm
from itertools import product

# --- CONFIGURAÇÕES DE SEGURANÇA ---
MAX_WORKERS = 3           # Reduz o número de requisições simultâneas
TIMEOUT_SECS = 30         # Aumenta o tempo de espera antes de desistir
PAUSA_MIN = 2.0           # Pausa mínima (segundos)
PAUSA_MAX = 5.0           # Pausa máxima (segundos)
MAX_TENTATIVAS = 3        # Número de repetições em caso de falha temporária
# ---------------------------------

def extrair_tabela(url):
    """
    Extrai a tabela de uma única URL com lógica de repetição e pausas.
    
    Retorna:
    - DataFrame em caso de sucesso.
    - String de erro em caso de falha.
    """
    for tentativa in range(1, MAX_TENTATIVAS + 1):
        # 1. Pausa Randômica (Anti-Banimento)
        pausa = random.uniform(PAUSA_MIN, PAUSA_MAX)
        time.sleep(pausa) 
        
        try:
            # 2. Requisição com Timeout Estendido
            response = requests.get(url, timeout=TIMEOUT_SECS)
            
            # 3. Verifica o Status Code
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                tabela = soup.find('table', class_='pagamentos')

                if tabela:
                    df = pd.read_html(StringIO(str(tabela)), thousands='.', decimal=',')[0]
                    
                    # Adiciona as colunas 'Ano' e 'Mês' para referência
                    params = url.split('?')[-1].split('&')
                    # Tenta extrair Ano e Mês, lidando com possíveis erros na URL
                    try:
                        ano = params[0].split('=')[-1]
                        mes = params[1].split('=')[-1]
                        df['Ano'] = ano
                        df['Mês'] = mes
                    except IndexError:
                        df['Ano'] = 'Desconhecido'
                        df['Mês'] = 'Desconhecido'
                    
                    return df # Retorna o DataFrame em caso de sucesso
                else:
                    return f"Tabela não encontrada: {url}"
            
            # Trata erros HTTP que não são Timeout (Ex: 404, 500)
            else:
                return f"ERRO HTTP {response.status_code} na tentativa {tentativa}/{MAX_TENTATIVAS}: {url}"
            
        except requests.exceptions.Timeout:
            # Trata o erro específico de Timeout
            if tentativa < MAX_TENTATIVAS:
                # Usa tqdm.write para logar a repetição sem quebrar a barra de progresso
                tqdm.write(f"⏳ Timeout na URL. Tentando novamente ({tentativa + 1}/{MAX_TENTATIVAS})...")
                time.sleep(5) # Pausa fixa maior antes de tentar de novo
                continue
            else:
                return f"FALHA FINAL CONEXÃO (Timeout): {url}"
                
        except requests.exceptions.RequestException as e:
            # Trata outros erros de Requisição (Ex: DNS, SSL)
            return f"FALHA GERAL CONEXÃO: {e} - {url}"

    return None # Nunca deve ser alcançado, mas garante que a função retorna algo


# --- GERAÇÃO DA LISTA DE URLs (Janeiro/2020 a Outubro/2025) ---

anos = range(2020, 2026)
meses = range(1, 13)
base_url = 'https://transparencia.alesc.sc.gov.br/deputados.php?'
urls_a_processar = []

for ano, mes in product(anos, meses):
    # Intervalo: de Jan/2020 até Out/2025
    if ano == 2025 and mes > 10:
        continue
    
    url_final = f'{base_url}ano={ano}&mes={mes}'
    urls_a_processar.append(url_final)

# --- EXECUÇÃO PARALELA COM TQDM ---

resultados = []
erros = []

print(f"Iniciando extração de {len(urls_a_processar)} meses com {MAX_WORKERS} workers e pausa randômica entre {PAUSA_MIN}s e {PAUSA_MAX}s.")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Mapeia a função para as URLs
    futures = [executor.submit(extrair_tabela, url) for url in urls_a_processar]
    
    # Itera sobre os resultados com tqdm
    for future in tqdm(futures, total=len(urls_a_processar), desc="Extraindo Dados por Mês"):
        resultado = future.result()
        
        if isinstance(resultado, pd.DataFrame):
            resultados.append(resultado)
        else:
            # Se não for DataFrame, é uma mensagem de erro ou URL de falha
            erros.append(resultado)
            tqdm.write(f"❌ Falha: {resultado}") 

# --- COMBINAÇÃO FINAL ---

if resultados:
    # 4. Combina todos os DataFrames em um único resultado
    df_final = pd.concat(resultados, ignore_index=True)
    
    print("\n" + "="*50)
    print("✅ EXTRAÇÃO COMPLETA E BEM-SUCEDIDA")
    print(f"Total de registros obtidos: {len(df_final)}")
    print("Primeiras 5 linhas do DataFrame final:")
    print(df_final.head())
    print("="*50)
else:
    print("\n❌ Nenhuma tabela foi extraída com sucesso.")

if erros:
    print(f"\n⚠️ Total de erros ({len(erros)}):")
    # Imprime os 5 primeiros erros para ter uma ideia
    for i, erro in enumerate(erros[:5]):
        print(f"  - {erro}")
    if len(erros) > 5:
        print(f"  ...e mais {len(erros) - 5} erros.")

Iniciando extração de 70 meses com 3 workers e pausa randômica entre 2.0s e 5.0s.


Extraindo Dados por Mês: 100%|██████████| 70/70 [02:52<00:00,  2.46s/it]


✅ EXTRAÇÃO COMPLETA E BEM-SUCEDIDA
Total de registros obtidos: 2961
Primeiras 5 linhas do DataFrame final:
             Nome Subsídio (R$) Licença (Ato Plenário) Imposto de Renda  \
0     ADA DE LUCA      25322.25                      -          -5909.7   
1    ALTAIR SILVA      25322.25                      -          -5909.7   
2  ANA CAMPAGNOLO      25322.25                      -          -5909.7   
3        PAULINHA      25322.25                      -          -5909.7   
4     BRUNO SOUZA      25322.25                      -          -5909.7   

  Contribuição Previdênciaria Referência   Ano Mês  
0                     -671.11    01/2020  2020   1  
1                     -671.11    01/2020  2020   1  
2                     -671.11    01/2020  2020   1  
3                     -671.11    01/2020  2020   1  
4                     -671.11    01/2020  2020   1  


In [6]:
import os

caminho = os.path.join('data', 'holerite_deputados_sc.csv')
df_final.to_csv(caminho)